In [1]:
%load_ext autoreload
%autoreload 2

import os
from os import listdir
from os.path import isfile, join

import numpy as np
import matplotlib.pyplot as plt
import madmom

import sys
sys.path.append('../src')
from preprocessing import get_dataset
from models import OLSPatchRegressor
import visualize
MUSIC = 1
SPEECH = 0

na = np.newaxis

plt.rc('text', usetex=True)
plt.rc('font', family='serif')

Using TensorFlow backend.
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [3]:
music_dir  = '../data/music_speech/music_wav/'
speech_dir = '../data/music_speech/speech_wav/'

max_samples = -1

X, Y = get_dataset(music_dir, speech_dir, hpool=0, wpool=0, 
                   num_samples=max_samples, shuffle=True, reload=False,
                   window=np.hanning, fps=100, num_bands=3, fmin=30, fmax=17000,
                   fft_sizes=[1024, 2048, 4096]
                  )
print('Train Set Shape')
print(X.shape, Y.shape)

Y = (Y + 1) / 2 

loaded from disk
Train Set Shape
(6000, 1024, 10, 3) (6000,)


# Train CNN using KERAS

In [4]:
batch_size = 8
num_classes = 2
epochs = 109

# --------------------------------------------
num_frequencies = X.shape[1]
num_timesteps   = X.shape[2]
num_channels    = X.shape[3]
filter_time_size = 3
input_shape = num_frequencies, num_timesteps, num_channels

# DEFINE MODEL
model = Sequential()
model.add(MaxPooling2D(pool_size=(1, 3), input_shape=input_shape))
model.add(Conv2D(32, kernel_size=(num_frequencies, filter_time_size),
                 activation='relu'))

model.add(Conv2D(1, kernel_size=(1, 1), activation='sigmoid'))
model.add(Lambda(lambda x: K.mean(x, axis=[1,2])))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [5]:
t_ind = 96
XTrain = X[:t_ind]
YTrain = Y[:t_ind]
XTest  = X[t_ind:]
YTest  = Y[t_ind:]

print(XTrain.shape, YTrain.shape)
print(XTest.shape, YTest.shape)

(96, 1024, 10, 3) (96,)
(5904, 1024, 10, 3) (5904,)


In [6]:
model.fit(XTrain, YTrain,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(XTest, YTest))
score = model.evaluate(XTest, YTest, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 96 samples, validate on 5904 samples
Epoch 1/109
96/96 [==============================] - 3s 34ms/step - loss: 0.6241 - acc: 0.6146 - val_loss: 0.6014 - val_acc: 0.6021
Epoch 2/109
96/96 [==============================] - 3s 28ms/step - loss: 0.2960 - acc: 0.8854 - val_loss: 0.2623 - val_acc: 0.9336
Epoch 3/109
96/96 [==============================] - 3s 35ms/step - loss: 0.1800 - acc: 0.9688 - val_loss: 0.2139 - val_acc: 0.9346
Epoch 4/109
96/96 [==============================] - 2s 25ms/step - loss: 0.1347 - acc: 0.9688 - val_loss: 0.1703 - val_acc: 0.9688
Epoch 5/109
96/96 [==============================] - 2s 25ms/step - loss: 0.1037 - acc: 0.9896 - val_loss: 0.1436 - val_acc: 0.9719
Epoch 6/109
96/96 [==============================] - 2s 25ms/step - loss: 0.0869 - acc: 0.9896 - val_loss: 0.1344 - val_acc: 0.9632
Epoch 7/109
96/96 [==============================] - 2s 25ms/step - loss: 0.0627 - acc: 0.9896 - val_loss: 0.1156 - val_acc: 0.9849
Epoch 8/109
96/96 [===========

96/96 [==============================] - 3s 28ms/step - loss: 4.5685e-05 - acc: 1.0000 - val_loss: 0.0145 - val_acc: 0.9959
Epoch 62/109
96/96 [==============================] - 2s 25ms/step - loss: 4.2914e-05 - acc: 1.0000 - val_loss: 0.0144 - val_acc: 0.9961
Epoch 63/109
96/96 [==============================] - 3s 28ms/step - loss: 4.0464e-05 - acc: 1.0000 - val_loss: 0.0144 - val_acc: 0.9961
Epoch 64/109
96/96 [==============================] - 3s 28ms/step - loss: 3.8223e-05 - acc: 1.0000 - val_loss: 0.0143 - val_acc: 0.9961
Epoch 65/109
96/96 [==============================] - 2s 26ms/step - loss: 3.6307e-05 - acc: 1.0000 - val_loss: 0.0143 - val_acc: 0.9961
Epoch 66/109
96/96 [==============================] - 2s 25ms/step - loss: 3.4433e-05 - acc: 1.0000 - val_loss: 0.0143 - val_acc: 0.9961
Epoch 67/109
96/96 [==============================] - 2s 25ms/step - loss: 3.2816e-05 - acc: 1.0000 - val_loss: 0.0142 - val_acc: 0.9961
Epoch 68/109
96/96 [==============================] - 

In [7]:
model_path = '../models/keras/'
os.makedirs(model_path, exist_ok=True)

model.save(os.path.join(model_path, 'cnn_on_spectros_filter{}-{:2.2f}.h5'.format(filter_size, score[1])))

NameError: name 'filter_size' is not defined

In [ ]:

music = XTest[YTest == MUSIC][0]
speech = XTest[YTest == SPEECH][1]
visualize.prediction_over_time(music, speech, model)